In [1]:
import os
import sys
from pathlib import Path

mod_path = os.path.join(Path.cwd().parent.parent)
if mod_path not in sys.path:
    sys.path.append(mod_path)

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold

from src.data import *
from src.features.utils import *
from src.model.tree_based import ModelXgBoost

In [3]:
# data with shape 28007, 33 [transaction related features]
train = pd.read_csv('../../data/processed/train.csv')

In [4]:
train.columns

Index(['Unnamed: 0', 'ID', 'Deposit', 'AccessoryRate', 'rateTypeEntity',
       'RatePerUnit', 'DaysOnDeposit', 'MainApplicantGender', 'Age', 'Region',
       'Occupation', 'Term', 'TotalContractValue', 'ExpectedTermDate',
       'FirstPaymentDate', 'LastPaymentDate', 'm1', 'm2', 'm3', 'm4', 'm5',
       'm6', 'SplitPaymentsHistory', 'nb_payments', 'amount_paid',
       'percent_amt_paid', 'mean_amt_paid', 'median_amt_paid', 'max_amt_paid',
       'min_amt_paid', 'stddev_amt_paid', 'b1', 'b2', 'b3', 'b4', 'b5'],
      dtype='object')

In [5]:
train.drop(columns = 'Unnamed: 0', inplace=True)
train.columns

Index(['ID', 'Deposit', 'AccessoryRate', 'rateTypeEntity', 'RatePerUnit',
       'DaysOnDeposit', 'MainApplicantGender', 'Age', 'Region', 'Occupation',
       'Term', 'TotalContractValue', 'ExpectedTermDate', 'FirstPaymentDate',
       'LastPaymentDate', 'm1', 'm2', 'm3', 'm4', 'm5', 'm6',
       'SplitPaymentsHistory', 'nb_payments', 'amount_paid',
       'percent_amt_paid', 'mean_amt_paid', 'median_amt_paid', 'max_amt_paid',
       'min_amt_paid', 'stddev_amt_paid', 'b1', 'b2', 'b3', 'b4', 'b5'],
      dtype='object')

In [6]:
train.shape

(28007, 35)

In [7]:
# Region has certain NaN values which might cause issues while encoding
# As total NaNs constitute ~5% of the data (1446) we remove it as of now
print(train['Region'].isna().sum() / train.shape[0] * 100)
train.dropna(subset=['Region'], how='all', inplace=True)

5.162994965544328


In [8]:
train.shape

(26561, 35)

In [9]:
# split_payment_history_df = train[["ID", "SplitPaymentsHistory"]]
id_arr = train[["ID"]]

target = train[['m1', 'm2', 'm3', 'm4', 'm5', 'm6']]
train_arr = train.drop(columns=['m1', 'm2', 'm3', 'm4', 'm5', 'm6', 
                                'ID',
                                'SplitPaymentsHistory',
                                'ExpectedTermDate', 
                                'FirstPaymentDate',
                                'LastPaymentDate',])

## Approach 1

### Model train on initial hp

In [10]:
def approach_one_model(x_train, y_train, x_test):
    model = ModelXgBoost(train_array=x_train, train_target=y_train)
    model.train_model()  # Default h.params (Checkout the code)
    
    predict = model.trained_model.predict(x_test)
    
    return model, predict

In [11]:
def encode_and_drop(full_array, data_type, tr_encoder=None):
#     print(tr_encoder)
    categorical_array = full_array[full_array.select_dtypes(exclude=['number']).columns]
    numerical_array = full_array.drop(columns=X_train.select_dtypes(exclude=['number']).columns)
#     print(numerical_array.shape)
    encoded_array, encoder = one_hot_encoding(
        categorical_frame=categorical_array, 
        type_of_data=data_type,
        fitted_encoder=tr_encoder,
        conv=True
    )
#     print(encoded_array.shape)
    final_array = pd.concat([numerical_array.reset_index(drop=True), 
                             encoded_array.reset_index(drop=True)], axis=1)
    final_array.index = numerical_array.index
#     print(final_array.shape)
    return numerical_array, encoded_array, encoder

In [12]:
# split data into train and test sets
from sklearn.model_selection import train_test_split

seed = 10
X_train, X_test, y_train, y_test = train_test_split(train_arr, target, test_size=0.45, random_state=seed)

In [13]:
model_dict = dict()
predict_dict = dict()
for model_no, target_col in enumerate(["m1", "m2", "m3", "m4", "m5", "m6"]):
    og_frame, encoded_train, encoder_model = encode_and_drop(X_train, "train", None)
    og_frame_test, encoded_test, encoder_model = encode_and_drop(X_test, "test", encoder_model)
    model_dict[f"M_{model_no}"], predict_dict[f"m{model_no+1}_pred"] = approach_one_model(
        x_train=encoded_train,
        y_train=y_train[[target_col]],
        x_test=encoded_test
    )

model_dict = dict()
predict_dict = dict()
for model_no, target_col in enumerate(["m1", "m2", "m3", "m4", "m5", "m6"]):
    og_frame, encoded_train, encoder_model = encode_and_drop(X_train, "train", None)
    og_frame_test, encoded_test, encoder_model = encode_and_drop(X_test, "test", encoder_model)
    model_dict[f"M_{model_no}"], predict_dict[f"m{model_no+1}_pred"] = approach_one_model(
        x_train=encoded_train,
        y_train=y_train[[target_col]],
        x_test=encoded_train
    )

In [15]:
predict_dict

{'m1_pred': array([809.8957 , 805.21423, 772.66815, ..., 809.8957 , 754.74084,
        707.98004], dtype=float32),
 'm2_pred': array([766.40015, 813.8415 , 788.25244, ..., 766.40015, 760.3801 ,
        991.70056], dtype=float32),
 'm3_pred': array([759.24927, 801.59204, 778.04877, ..., 759.24927, 762.9633 ,
        729.9201 ], dtype=float32),
 'm4_pred': array([774.0879 , 830.2388 , 760.61194, ..., 774.0879 , 748.584  ,
        789.7673 ], dtype=float32),
 'm5_pred': array([811.49286, 823.3398 , 778.02075, ..., 811.49286, 753.9366 ,
        753.0577 ], dtype=float32),
 'm6_pred': array([616.63574, 760.2139 , 703.4711 , ..., 616.63574, 605.63354,
        873.1166 ], dtype=float32)}

### Calculation of RMSE

In [17]:
pred_frame = pd.DataFrame(predict_dict)
pred_frame.index = X_test.index
# pred_frame.index = X_train.index

In [18]:
full_test_array = pd.concat([X_test, y_test, pred_frame], axis=1)
# full_test_array = pd.concat([X_train, y_train, pred_frame], axis=1)

In [19]:
full_test_array = pd.merge(full_test_array, id_arr, how='left', left_index=True, right_index=True)

In [20]:
sub_file = SubmissionFile(
    validation_data=full_test_array,
    type_of_data='validation'
).execute()

In [21]:
sub_file.shape

(87648, 3)

In [22]:
sub_file['SquaredError'] = np.square(sub_file['Target'] - sub_file['Prediction'])

In [23]:
rmse = np.sqrt(np.sum(sub_file['SquaredError'])/sub_file.shape[0])
print('Final RMSE --> ', rmse)

Final RMSE -->  906.6688076344569
